In [ ]:
import pandas as pd

In [ ]:
exports_to_analyze = ['grassen-1']
all_df = pd.DataFrame()

# Load the data
for export in exports_to_analyze:
    filename = 'export/' + export + '/latency_per_publishers.csv'
    df = pd.read_csv(filename, sep=',', index_col=0)
    all_df = pd.concat([all_df, df])

In [ ]:
threshold = 25

# Filter out the data
all_df = all_df[all_df['nb_publishers'] <= threshold]

df = all_df.groupby('case').mean()
# calculate the overhead against native
df['overhead_against_native'] = (df['latency']) / df.loc['native-tls', 'latency']
df['overhead_against_wasm'] = (df['latency']) / df.loc['wasm-tls', 'latency']

# round the overhead values to 2 digits
df['overhead_against_native'] = df['overhead_against_native'].round(2)
df['overhead_against_wasm'] = df['overhead_against_wasm'].round(2)
df

In [ ]:
message_df = pd.read_csv('export/' + exports_to_analyze[0] + '/message_latency.csv', sep=',', index_col=0)
# get unique values for 'case' column
cases = ['native-tls', 'wasm-tls', 'sgx-tls']
message_df = message_df.drop(columns=['timestamp_receive', 'timestamp_send'])
# get unique values for 'nb_publishers' column
nb_publishers = message_df['nb_publishers'].unique()

for case in cases:
    export_key_values = pd.DataFrame()
    export_outliers = pd.DataFrame()
    for nb_publisher in nb_publishers:
        df = message_df[(message_df['case'] == case) & (message_df['nb_publishers'] == nb_publisher)]
        df = df.rename(columns={'latency': nb_publisher})
        df = df.drop(columns=['case', 'nb_publishers'])

        export = pd.DataFrame()
        export['q1'] = df.quantile(0.25)
        export['q2'] = df.median()
        export['q3'] = df.quantile(0.75)
        iqr = export['q3'] - export['q1']
        export['lower_whisker'] = max(df.min()[0], (export['q1'] - 1.5 * iqr)[0])
        export['upper_whisker'] = min(df.max()[0], (export['q3'] + 1.5 * iqr)[0])
        export['max'] = df.max()
        export['min'] = df.min()

        export = export.reset_index()
        export = export.rename(columns={'index': 'nb_publishers'})

        # get outliers
        outliers = df[(df < export['lower_whisker'][0]) | (df > export['upper_whisker'][0])].dropna().reset_index(drop=True)
        export = export.reset_index(drop=True)
        export_outliers = pd.concat([export_outliers, outliers], axis=1)

        export_key_values = pd.concat([export_key_values, export])


    export_key_values.to_csv('export/' + exports_to_analyze[0] + '/message_latency_' + str(case) + '.csv')
    export_outliers.to_csv('export/' + exports_to_analyze[0] + '/message_latency_outliers_' + str(case) + '.csv')
